### Midterm Practice

### Getting a Data Frame containing player stats

In [5]:
import pandas as pd
import requests
from typing import Optional, Dict

KEY = "82c2ff00a706bf9dd19cdd152fd01aeb"
URL = "https://v3.football.api-sports.io/"

def call_api(endpoint: str, params: Optional[Dict] = None) -> pd.DataFrame:
    """
    Makes a GET request to the API endpoint with optional parameters.
    Returns a Pandas DataFrame with flattened structure if the response contains data.
    Handles Errors and Exceptions.
    """

    params = params or {}
    headers = {"x-rapidapi-key": KEY}
    url = f"{URL}{endpoint}"
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status() # Raises an error for bad responses
        data = response.json()
        
        if not data.get("response"):
            raise ValueError("No data found in the API response.")
            
        # Normalize the main response
        df = pd.json_normalize(data["response"], sep="_")
        
        # If statistics exist, process them
        if "statistics" in df.columns:
            # Reset index before exploding to avoid index issues
            df = df.reset_index(drop=True)
            
            # Explode statistics (each player might have multiple entries)
            df = df.explode("statistics", ignore_index=True)
            
            # Normalize statistics
            stats_df = pd.json_normalize(
                df["statistics"],
                sep="_",
                meta_prefix="statistics_"
            ).reset_index(drop=True)
            
            # Merge with player data
            df = pd.concat([
                df.drop("statistics", axis=1).reset_index(drop=True), 
                stats_df
            ], axis=1)
            
            # Flatten remaining nested columns
            for col in df.columns:
                if df[col].notna().any() and isinstance(df[col].iloc[0], dict):
                    nested_df = pd.json_normalize(df[col], sep="_").add_prefix(f"{col}_")
                    df = pd.concat([
                        df.drop(col, axis=1).reset_index(drop=True),
                        nested_df.reset_index(drop=True)
                    ], axis=1)
        
        return df
    
    except requests.exceptions.RequestException as e:
        raise Exception(f"API request failed: {str(e)}")
    except ValueError as ve:
        raise Exception (f"Valure Error : {str(ve)}")
    except Exception as e:
        raise Exception(f"Error processing API response: {str(e)}")


def clean_column_names(df):
    """
    Correctly rename columns while preserving:
    - Team fields (team_id, team_name)
    - Competition fields (competition_id, competition_name)
    - Player fields (player_id, player_name)
    """
    # Step 1: Remove prefixes carefully
    df.columns = df.columns.str.replace(r'^(statistics_|games_)', '', regex=True)  # Only remove unnecessary prefixes
    
    # Step 2: Explicitly rename key columns (no guessing!)
    column_renames = {
        # Player
        'player_id': 'player_id',
        'player_name': 'player_name',
        
        # Team
        'team_id': 'team_id',
        'team_name': 'team_name',
        'team_logo': 'team_logo',
        
        # Competition
        'league_id': 'competition_id',
        'league_name': 'competition_name',
        'league_logo': 'competition_logo',
        'country_logo': 'country_flag',  # Rename for clarity
    }
    
    # Step 3: Apply renames (skip missing columns)
    return df.rename(columns={k: v for k, v in column_renames.items() if k in df.columns})

def get_player_stats(player_id: str, season: str) -> pd.DataFrame:
    """Fetches and prints player stats for a given season."""
    params = {"id": player_id, "season": season}
    df = call_api("players", params)
    df = clean_column_names(df)

    print("\nPlayer Statistics:")
    print(df.to_string(index=False))  # Print without DataFrame index
    return df

if __name__ == "__main__":
    try:
        df = get_player_stats("133609", "2023")
        print(df["shots_total"].sum())
    except Exception as e:
        print(f"Error: {str(e)}")
    


Player Statistics:
 player_id player_name player_firstname player_lastname  player_age player_birth_date player_birth_place player_birth_country player_nationality player_height player_weight  player_injured                                            player_photo  team_id team_name                                          team_logo  competition_id                  competition_name league_country                                     competition_logo                              league_flag  league_season  appearences  lineups  minutes number   position   rating  captain  substitutes_in  substitutes_out  substitutes_bench  shots_total  shots_on  goals_total  goals_conceded  goals_assists goals_saves  passes_total  passes_key  passes_accuracy  tackles_total  tackles_blocks  tackles_interceptions  duels_total  duels_won  dribbles_attempts  dribbles_success dribbles_past  fouls_drawn  fouls_committed  cards_yellow  cards_yellowred  cards_red penalty_won penalty_commited  penalty_scored  pen

### Initializing a Player Class

In [ ]:
class Player: 
    def __init__(self, player_name, age, height, weight, team_name, games_appearances, 
                 games_minutes, games_position, games_rating, shots_total, shots_on, 
                 goals_total, goals_conceded, goals_assists, goals_saves, passes_total, 
                 passes_key, passes_accuracy, tackles_total, tackles_blocks, tackles_interceptions, 
                 duels_total, duels_won, dribbles_attemtps, dribbles_success, dribbles_past, 
                 fouls_drawn, fouls_committed, cards_yellow, cards_yellowred, cards_red, 
                 penalty_won, penalty_commited, penalty_scored, penalty_missed, penalty_saved):
        
        self.player_name = player_name
        self.age = age
        self.height = height
        self.weight = weight
        self.team_name = team_name
        self.games_appearances = games_appearances
        self.games_minutes = games_minutes
        self.games_position = games_position
        self.games_rating = games_rating
        self.shots_total = shots_total
        self.shots_on = shots_on
        self.goals_total = goals_total
        self.goals_conceded = goals_conceded
        self.goals_assists = goals_assists
        self.goals_saves = goals_saves
        self.passes_total = passes_total
        self.passes_key = passes_key
        self.passes_accuracy = passes_accuracy
        self.tackles_total = tackles_total
        self.tackles_blocks = tackles_blocks
        self.tackles_interceptions = tackles_interceptions
        self.duels_total = duels_total
        self.duels_won = duels_won
        self.dribbles_attemtps = dribbles_attemtps
        self.dribbles_success = dribbles_success
        self.dribbles_past = dribbles_past
        self.fouls_drawn = fouls_drawn
        self.fouls_committed = fouls_committed
        self.cards_yellow = cards_yellow
        self.cards_yellowred = cards_yellowred
        self.cards_red = cards_red
        self.penalty_won = penalty_won
        self.penalty_commited = penalty_commited
        self.penalty_scored = penalty_scored
        self.penalty_missed = penalty_missed
        self.penalty_saved = penalty_saved


Pedri = Player(self, )